In [ ]:
from bokeh.plotting import figure, show, output_file
from bokeh.tile_providers import CARTODBPOSITRON
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral6
from bokeh.transform import linear_cmap

import math
import pandas as pd
import numpy as np


output_notebook()

In [ ]:
full_path = 'Japan storms all years.csv'
tracks = pd.read_csv(full_path, sep=';')
tracks = tracks[tracks['Time of analysis'] != 66666]
print(tracks.head())
#print(tracks.info())

In [ ]:
def merc(lat, lon, unit=0.1):
    lat =lat * 0.1
    lon = lon * 0.1
    r_major = 6378137.000
    x = r_major * np.deg2rad(lon)
    scale = x/lon
    y = 180.0/math.pi * np.log(np.tan(math.pi/4.0 + 
        lat * (math.pi/180.0)/2.0)) * scale
    return (x, y)

scaled_lat,scaled_lon = merc(tracks.Lat.values, tracks.Lon.values)
tracks['scaled_lat'] = scaled_lat
tracks['scaled_lon'] = scaled_lon
tracks['Max sustained wind speed km_per_h'] = tracks['Max sustained wind speed'] * 1.852

In [ ]:
TOOLTIPS = [
    ("Name", "@name"),
   # ("(lat,lon)", "(@lat, @lon)"),
    ("Central Pressure", "@pressure"),
    ("Category", "@category"),
    #("Max sustained wind speed in knot", "@max_wind_speed")
    ("Max sustained wind speed in km/h", "@max_wind_speed")
]
p = figure(x_range=(14000000, 16000000), y_range=(1500000, 7000000),
           x_axis_type="mercator", y_axis_type="mercator",tools='pan,hover,wheel_zoom',tooltips=TOOLTIPS)
p.add_tile(CARTODBPOSITRON)

In [ ]:
mapper = linear_cmap(field_name='category', palette=Spectral6, 
                         low=min(tracks['Grade'].values),
                         high=max(tracks['Grade'].values))

In [ ]:
paths = tracks.groupby('Name')
print(paths.groups.keys())
#print(paths.get_group('JEBI'))

In [ ]:
#storms_to_plot = paths.groups.keys()
storms_to_plot = ['JEBI_1821', 'TRAMI_1824', 'VERA_5915','SONGDA_418','MIREILLE_9119']

In [ ]:
for storm in storms_to_plot:
    path = paths.get_group(storm)
    source = ColumnDataSource(
        data=dict(lon=path.scaled_lon.values, lat=path.scaled_lat.values, pressure=path['Central Pressure'].values,
                  name=path.Name.values, category=path.Grade.values, max_wind_speed=path['Max sustained wind speed km_per_h'].values)
    )

    p.line(x="lat", y="lon", line_color="blue", line_alpha=0.8, source=source)
    p.circle(x="lat", y="lon", size=7, fill_color=mapper,color=mapper, fill_alpha=0.8, source=source)
    #fill_color="blue", size="category"
show(p)